# Entrenamiento Completo para Dueling DQN

## Librerias

In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from gym_envs import make_galaxian_env
from utils import set_global_seed, get_seed
from models import DuelingDQN
from replay_buffer import ReplayBuffer
from policies import DQNPolicy

import matplotlib.pyplot as plt

## Configuración de CUDA

In [2]:
seed = get_seed()
if seed is not None:
    print(f"Usando SEED global = {seed}")
    set_global_seed(seed)
else:
    # Crear una semilla aleatoria y usarla
    seed = np.random.randint(0, 10000)
    set_global_seed(seed)

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("Using CPU")

Usando SEED global = 42
Using GPU: NVIDIA GeForce RTX 3070 Ti Laptop GPU


## Helper para convertir obs → estado (C, H, W)

In [3]:
def obs_to_state(obs) -> np.ndarray:
    """
    Convierte la observación del entorno (que puede venir como LazyFrames
    u otro tipo) a un np.ndarray con shape (C, H, W).
    Args:
        obs: observación del entorno
    Returns:
        np.ndarray con shape (C, H, W)
    """
    arr = np.array(obs, copy=False)

    if arr.ndim == 3:
        # Casos típicos:
        # - (H, W, C)
        # - (C, H, W)
        H, W = 84, 84  # por diseño de AtariPreprocessing

        if arr.shape[0] == H and arr.shape[1] == W:
            # (H, W, C) -> (C, H, W)
            arr = np.transpose(arr, (2, 0, 1))
        elif arr.shape[1] == H and arr.shape[2] == W:
            # ya está en (C, H, W), no tocamos
            pass
        else:
            raise ValueError(f"Shape de obs inesperado: {arr.shape}")
    elif arr.ndim == 2:
        # 1 frame (H, W) -> (1, H, W)
        arr = arr[None, ...]
    else:
        raise ValueError(f"Obs con ndim no soportado: {arr.ndim}")

    return arr

## Función de entrenamiento

In [ ]:
def train_dqn_variant(
    model_class,                 # DQN o DuelingDQN
    num_episodes: int,
    max_steps: int,
    label: str = "DQN",
    seed_base: int | None = None,
    print_every: int = 10,
    checkpoint_every: int = 100,
):
    """
    Entrena un modelo DQN-like (DQN o DuelingDQN) en Galaxian.
    Crea:
        - nuevo env
        - nuevo modelo online/target
        - nuevo replay buffer
        - su propio loop de entrenamiento

    Args:
        - model_class: clase del modelo a usar (DQN o DuelingDQN)
        - num_episodes: número de episodios para entrenar
        - max_steps: número máximo de pasos por episodio
        - label: etiqueta para identificar la variante en los logs
        - seed_base: semilla base para reproducibilidad (opcional)

    Returns:
        - lista de recompensas por episodio
        - lista de longitudes de episodio
    """
    # Crear entorno DeepMind preprocesado
    env = make_galaxian_env(
        render_mode=None,           # para entrenamiento, más rápido
        deepmind_wrappers=True,     # aplica AtariPreprocessing + FrameStackObservation
        frame_stack=4,              # número de frames a apilar
    )

    # Reset inicial para definir input shape
    if seed_base is not None:
        obs, _ = env.reset(seed=seed_base)
    else:
        obs, _ = env.reset()

    state = obs_to_state(obs)
    in_channels = state.shape[0]
    num_actions = env.action_space.n

    # Redes online y target
    online_net = model_class(in_channels=in_channels, num_actions=num_actions).to(device)
    target_net = model_class(in_channels=in_channels, num_actions=num_actions).to(device)
    target_net.load_state_dict(online_net.state_dict())
    target_net.eval()

    # Optimizador
    optimizer = optim.Adam(online_net.parameters(), lr=1e-4)

    # Hiperparámetros
    gamma = 0.99
    batch_size = 32

    # Buffer de replay grande para estabilidad
    buffer_capacity = 350_000

    # Empezar a aprender cuando ya haya suficiente diversidad en el buffer
    start_learning_after = 50_000

    # Frecuencia de actualización de la red target
    target_update_freq = 10_000

    # Decaimiento de epsilon
    epsilon_start = 1.0
    epsilon_end = 0.1
    epsilon_decay_steps = 1_000_000

    # Replay buffer
    obs_shape = state.shape
    replay_buffer = ReplayBuffer(
        capacity=buffer_capacity,
        obs_shape=obs_shape,
        device=device,
        dtype=np.uint8,
    )

    print(f"[{label}] max_steps={max_steps}")
    print(f"[{label}] buffer_capacity={buffer_capacity}")
    print(f"[{label}] start_learning_after={start_learning_after}")
    print(f"[{label}] target_update_freq={target_update_freq}")
    print(f"[{label}] epsilon_decay_steps={epsilon_decay_steps}")

    def epsilon_by_step(step: int) -> float:
        """
        Calcula el valor de epsilon para el paso dado.
        Args:
            step: paso global actual
        Returns:
            valor de epsilon
        """
        if step >= epsilon_decay_steps:
            return epsilon_end
        else:
            return epsilon_start - (epsilon_start - epsilon_end) * (step / epsilon_decay_steps)

    def select_action(state: np.ndarray, step: int) -> int:
        """
        Selecciona una acción usando una política epsilon-greedy.
        Args:
            state: estado actual
            step: paso global actual
        Returns:
            acción seleccionada
        """
        eps = epsilon_by_step(step)
        if np.random.rand() < eps:
            return np.random.randint(0, num_actions)
        else:
            state_t = torch.as_tensor(state, device=device).unsqueeze(0)
            with torch.no_grad():
                q_values = online_net(state_t)
                action = int(q_values.argmax(dim=1).item())
            return action

    def dqn_update():
        """
        Realiza una actualización del modelo DQN usando un batch de la replay buffer.
        Returns:
            valor de la pérdida (loss) o None si no se puede actualizar
        """
        if len(replay_buffer) < batch_size:
            return None

        states, actions, rewards, next_states, dones = replay_buffer.sample(batch_size)

        q_values = online_net(states)
        q_sa = q_values.gather(1, actions.unsqueeze(1)).squeeze(1)

        with torch.no_grad():
            next_q_values = target_net(next_states)
            max_next_q = next_q_values.max(dim=1).values
            target = rewards + gamma * (1.0 - dones) * max_next_q

        loss = nn.SmoothL1Loss()(q_sa, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        return loss.item()

    # Loop de entrenamiento
    global_step = 0
    rewards_per_episode = []
    episode_lengths = []

    print(f"\n[TRAIN] Iniciando entrenamiento para {label}")

    for episode in range(1, num_episodes + 1):
        # si queremos reproducibilidad entre variantes, podemos desplazar la seed
        if seed_base is not None:
            obs, _ = env.reset(seed=seed_base + episode)
        else:
            obs, _ = env.reset()

        state = obs_to_state(obs)
        done = False
        episode_reward = 0.0
        steps_in_episode = 0

        if global_step >= max_steps:
            print(f"[{label}] Alcanzado max_steps={max_steps}. Fin del entrenamiento.")
            break

        while not done and global_step < max_steps:
            action = select_action(state, global_step)
            next_obs, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated

            r_env = reward
            r_clip = np.clip(r_env, -1.0, 1.0)

            next_state = obs_to_state(next_obs)

            replay_buffer.add(
                state=state,
                action=action,
                reward=r_clip,
                next_state=next_state,
                done=done,
            )

            state = next_state
            episode_reward += r_env
            global_step += 1
            steps_in_episode += 1

            if global_step > start_learning_after:
                _ = dqn_update()

            if global_step % target_update_freq == 0:
                target_net.load_state_dict(online_net.state_dict())

            if global_step >= max_steps:
                done = True
                break

        rewards_per_episode.append(episode_reward)
        episode_lengths.append(steps_in_episode)
        
        if episode % print_every == 0 or episode == 1 or global_step >= max_steps:
            print(
                f"[{label}] Ep {episode:4d} | Step {global_step:7d} | "
                f"Reward: {episode_reward:5.1f} | Eps: {epsilon_by_step(global_step):.3f}"
            )
        
        if episode % checkpoint_every == 0 or global_step >= max_steps or episode == 1:
            ckpt_path = os.path.join(
                    '../checkpoints/',
                    f"{label}_ep{episode:04d}_step{global_step}.pt",
                )
            torch.save(
                    {
                        "model_state_dict": online_net.state_dict(),
                        "optimizer_state_dict": optimizer.state_dict(),
                        "global_step": global_step,
                        "episode": episode,
                        "gamma": gamma,
                        "batch_size": batch_size,
                        "buffer_capacity": buffer_capacity,
                        "start_learning_after": start_learning_after,
                        "target_update_freq": target_update_freq,
                        "epsilon_start": epsilon_start,
                        "epsilon_end": epsilon_end,
                        "epsilon_decay_steps": epsilon_decay_steps,
                    },
                    ckpt_path,
                )

    env.close()
    return rewards_per_episode, episode_lengths, online_net


## Loop de Entrenamiento

In [5]:
num_episodes = 1_000_000            # para pruebas
max_steps = 1_500_000               # steps totales por variante

In [6]:
rewards_dueling, len_dueling, final_model_dueling = train_dqn_variant(
    model_class=DuelingDQN,
    num_episodes=num_episodes,
    max_steps=max_steps,
    label="DuelingDQN_Final",
    seed_base=seed,
    print_every=10,
    checkpoint_every=100,
)

[DuelingDQN_Final] max_steps=1500000
[DuelingDQN_Final] buffer_capacity=1000000
[DuelingDQN_Final] start_learning_after=50000
[DuelingDQN_Final] target_update_freq=10000
[DuelingDQN_Final] epsilon_decay_steps=900000

[TRAIN] Iniciando entrenamiento para DuelingDQN_Final
[DuelingDQN_Final] Ep    1 | Step     273 | Reward: 280.0 | Eps: 1.000
[DuelingDQN_Final] Ep   10 | Step    6226 | Reward: 630.0 | Eps: 0.993
[DuelingDQN_Final] Ep   20 | Step   12401 | Reward: 560.0 | Eps: 0.987
[DuelingDQN_Final] Ep   30 | Step   18552 | Reward: 870.0 | Eps: 0.980
[DuelingDQN_Final] Ep   40 | Step   23532 | Reward: 300.0 | Eps: 0.975
[DuelingDQN_Final] Ep   50 | Step   28860 | Reward: 1400.0 | Eps: 0.970
[DuelingDQN_Final] Ep   60 | Step   34361 | Reward: 1280.0 | Eps: 0.964
[DuelingDQN_Final] Ep   70 | Step   39972 | Reward: 540.0 | Eps: 0.958
[DuelingDQN_Final] Ep   80 | Step   44293 | Reward: 720.0 | Eps: 0.953
[DuelingDQN_Final] Ep   90 | Step   51255 | Reward: 890.0 | Eps: 0.946
[DuelingDQN_Final

KeyboardInterrupt: 

In [ ]:
torch.save(final_model_dueling.state_dict(), "../models/dueling_dqn_final.pt")
print("Modelo DuelingDQN final guardado.")

## Graficos

In [ ]:
def moving_average(x, window):
    x = np.array(x, dtype=np.float32)
    if len(x) < window:
        return x
    return np.convolve(x, np.ones(window) / window, mode="valid")

In [ ]:
window = 20

ma_duel = moving_average(rewards_dueling, window)

plt.figure(figsize=(12, 6))

# curvas “crudas” con poca alpha
plt.plot(rewards_dueling, label="DuelingDQN (crudo)", alpha=0.25)

# medias móviles desplazadas para que caigan sobre el episodio correcto
episodes_ma_duel = np.arange(window - 1, window - 1 + len(ma_duel))
plt.plot(episodes_ma_duel, ma_duel, label=f"DuelingDQN (media {window})", linewidth=2)

plt.xlabel("Episodio")
plt.ylabel("Reward total")
plt.title("Reward por episodio con media móvil")
plt.legend()
plt.tight_layout()
plt.show()

## Desempeño

In [ ]:
policy = DQNPolicy(final_model_dueling, device=device, epsilon=0.0)
scores = []
for _ in range(50):
    env_eval = make_galaxian_env(deepmind_wrappers=True, frame_stack=4)
    obs, info = env_eval.reset()
    done = False
    total_r = 0
    while not done:
        action = policy(obs, info, env_eval.action_space)
        obs, r, terminated, truncated, info = env_eval.step(action)
        done = terminated or truncated
        total_r += r
    scores.append(total_r)
    env_eval.close()

print("Mean eval score DQN:", np.mean(scores))